In [ ]:
import cv2
import numpy as np
import os

# Load the image
image_path = './1.jpg'
assert os.path.exists(image_path)

img = cv2.imread(image_path)

In [ ]:
import cv2
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.convertScaleAbs(gray)
median_blur = cv2.medianBlur(gray, 5)
plt.imshow(cv2.cvtColor(median_blur, cv2.COLOR_BGR2RGB))

edges = cv2.Canny(median_blur, 100, 200)
plt.imshow(cv2.cvtColor(edges, cv2.COLOR_BGR2RGB))

In [ ]:
ret, thresh = cv2.threshold(edges, 127, 255, 0)
plt.imshow(cv2.cvtColor(thresh, cv2.COLOR_BGR2RGB))
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# Find the biggest contour
print(len(contours))

cv2.drawContours(img, contours, -1, (0,255,0), 3)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:


# Create an empty list to store all vertices and faces
all_vertices = []
all_faces = []

# Iterate over all contours
for contour in contours:
    # Extrude the contour to give it depth
    depth = 10  # Adjust as needed
    extruded_contour = np.dstack((contour[:, 0, 0], contour[:, 0, 1], np.zeros_like(contour[:, 0, 0]))).reshape((-1, 3))

    # Create the mesh for the current contour
    vertices = np.vstack((extruded_contour, extruded_contour + [0, 0, depth]))
    faces = []

    # Triangulate the faces for the current contour
    for i in range(len(extruded_contour) - 1):
        faces.append([i, i + 1, i + len(extruded_contour)])
        faces.append([i + 1, i + len(extruded_contour) + 1, i + len(extruded_contour)])

    # Connect the extruded contour to fill the space and create a solid volume
    for i in range(len(extruded_contour) - 1):
        faces.append([i, i + 1, i + len(extruded_contour)])
        faces.append([i + 1, i + len(extruded_contour) + 1, i + len(extruded_contour)])
        faces.append([i, i + len(extruded_contour), i + len(extruded_contour) + 1])
        faces.append([i, i + len(extruded_contour) + 1, i + 1])

    # Append the vertices and faces of the current contour to the lists
    all_vertices.append(vertices)
    all_faces.append(np.array(faces))

# Concatenate all vertices and faces
all_vertices = np.concatenate(all_vertices)
cumulative_vertex_count = np.cumsum([0] + [len(v) for v in all_vertices[:-1]])

all_faces_concatenated = []
for i, faces in enumerate(all_faces):
    all_faces_concatenated.append(faces + cumulative_vertex_count[i])

all_faces_concatenated = np.concatenate(all_faces_concatenated)

# Create the mesh
stl_mesh = mesh.Mesh(np.zeros(len(all_faces_concatenated), dtype=mesh.Mesh.dtype))
for i, f in enumerate(all_faces_concatenated):
    for j in range(3):
        stl_mesh.vectors[i][j] = all_vertices[f[j], :]

# Save the mesh as an STL file
stl_mesh.save('output_solid_all_contours.stl')
